reference:
- https://www.kaggle.com/code/pelinkeskin/deep-rl-with-stable-baseline3-and-gymnasium-ppo

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import stable_baselines3 as sb3
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
from stable_baselines3.common.monitor import load_results
from stable_baselines3.common.torch_layers import NatureCNN
from stable_baselines3.common.policies import ActorCriticPolicy, ActorCriticCnnPolicy
from stable_baselines3.common.env_checker import check_env
from kaggle_environments import make, evaluate

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [ ]:
LOG_DIR = os.path.join(os.getcwd(), 'log')	# トレーニングのログを保存するディレクトリ
os.makedirs(LOG_DIR, exist_ok=True)

MODEL_DIR = os.path.join(os.pardir, 'models')	# トレーニング済みモデルを保存するディレクトリ
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_PATH = os.path.join(MODEL_DIR, 'connectx_model')	# トレーニング済みモデルのパス

環境

In [ ]:
# 環境の作成
from environment import ConnectFourGym

training_env = ConnectFourGym(opponent='random')
training_env

In [ ]:
check_env(training_env, warn=True)

In [ ]:
# ログを取得する
training_env = Monitor(training_env, LOG_DIR, allow_early_resets=True)
training_env

In [ ]:
# 「DummyVecEnv」は、OpenAI Gymの環境をベクトル化するための特殊なラッパーです。
# 通常、強化学習アルゴリズムは一度に1つの環境しか処理できませんが、これを使用することで
# 複数の環境を同時に実行することができます。これにより、学習プロセスが効率的になります。
training_env = DummyVecEnv([lambda: training_env])
training_env

In [ ]:
training_env.observation_space.sample()

Training my vector agent with SB3 PPO Algorithm

In [ ]:
#code ref: https://github.com/araffin/rl-baselines-zoo/blob/master/utils/utils.py#L225
def liner_schedule(initial_value: float):
	"""
	Linear learning rate schedule.
	:param initial_value: (float)
	:return: (function)
	"""
	def func(progress_remaining: float) -> float:
		"""
		Progress will decrease from 1 (beginning) to 0
		:param progress_remaining: (float)
		:return: (float)
		"""
		return progress_remaining * initial_value
	return func

このPPOエージェントのアーキテクチャはpolicy='MlpPolicy'という引数によって指定されています。ここで'MlpPolicy'はMulti-Layer Perceptron(MLP、多層パーセプトロン)を使用することを意味します。

この'policy'引数は、エージェントが取るべき行動を決定する際に使用するニューラルネットワークの形状や種類を決定します。'MlpPolicy'は全結合のニューラルネットワークを使用します。

このネットワークの詳細なアーキテクチャ（例えば層の数、ノードの数、活性化関数等）は、ライブラリ（Stable Baselines3）内部で定義されています。具体的な設定を変更したい場合はpolicy_kwargs引数を使用してカスタマイズすることが可能です。

また、特定の問題に対して特殊なネットワークアーキテクチャを使用したい場合は、自分で定義したカスタムポリシーを作成し、そのポリシーを使用することも可能です。これは例えば、畳み込みニューラルネットワーク(CNN)やリカレントニューラルネットワーク(RNN)を使用したい場合などに有用です。

In [ ]:

if os.path.exists(MODEL_PATH):
    print('Loading existing model...')
    agent = PPO.load(MODEL_PATH, env=training_env, verbose=0)
else:
    print('Training new model...')
    agent = PPO(
        policy='MlpPolicy',	# ネットワークアーキテクチャ
        env=training_env,
        n_steps=1536,
        ent_coef=0.001,	# この値が大きいほど、エージェントはさまざまなアクションを試行する傾向があります
        n_epochs=8,
        gae_lambda=0.95,	# Generalized Advantage Estimator。報酬の割引率を制御し、エージェントが将来の報酬にどれだけ価値を置くかを調節
        learning_rate=liner_schedule(3e-4),
        batch_size=512,
        clip_range=0.4,	# PPOのクリップ範囲。PPOは勾配の更新を制限（クリップ）することで、学習の安定性を向上させます
        policy_kwargs={
            'log_std_init': -2,	# ログスケールでの標準偏差の初期値
            'ortho_init': False,	# 直交初期化の有無
        },
        verbose=1
    )

In [ ]:
print(agent.policy)

In [ ]:
eval_env = ConnectFourGym()
eval_env = Monitor(eval_env, LOG_DIR)
eval_env = DummyVecEnv([lambda: eval_env])

eval_callback = EvalCallback(eval_env,
                             best_model_save_path=LOG_DIR,
                             log_path=LOG_DIR,
                             eval_freq=1000,
                             render=False)

In [ ]:
for key, p in agent.get_parameters()['policy'].items():
    print(key, p.numel())
print(f"Total number of trainable parameters: {sum(p.numel() for ey, p in agent.get_parameters()['policy'].items())}")

In [ ]:
# Train the model for a large number of timesteps
agent.learn(
    total_timesteps=50000,
    reset_num_timesteps=True,
    callback=eval_callback
)

In [ ]:
evaluation_lof_file = os.path.join(LOG_DIR, 'evaluations.npz')
evaluation_log = np.load(evaluation_lof_file)
df_evaluation_log = pd.DataFrame({item: [np.mean(ep) for ep in evaluation_log[item]] for item in evaluation_log.files})

df_evaluation_log.head()

In [ ]:
ax = df_evaluation_log.loc[0: len(df_evaluation_log), 'results'].plot(color='lightgray', xlim=[0, len(df_evaluation_log)], figsize = (10,5))
df_evaluation_log['results'].rolling(5).mean().plot(color='black', xlim=[0, len(df_evaluation_log)])
ax.set_xticklabels(df_evaluation_log['timesteps'])
ax.set_xlabel("Eval Episode")
plt.ylabel("Rolling Mean Cumulative Return")
plt.show()

In [ ]:
# まず、agent.set_env(eval_env)はエージェントの評価環境を設定します。エージェントはeval_envという環境で評価されることになります。これは通常、トレーニング環境とは異なり、より一般的な環境か、または一部がランダム化された環境であることが多いです。
# 次にmean_reward, std_reward = sb3.common.evaluation.evaluate_policy(agent, agent.get_env(), n_eval_episodes=30)についてです。この行は、評価環境上でのエージェントの性能を評価します。具体的には、エージェントがn_eval_episodes=30（つまり30回のエピソード）でどの程度の報酬を得られるかを評価します。これはエージェントが新たな環境でどの程度うまく動くか（つまりどの程度汎用性があるか）を評価するために行われます。
# 得られた結果は、30エピソードの平均報酬（mean_reward）と報酬の標準偏差（std_reward）として出力されます。これらの値が高いほど、エージェントは評価環境で高い報酬を得られることが確かであり、したがってエージェントの性能が良いと言えます。
# 最後に、print("Mean Reward: {} +/- {}".format(mean_reward, std_reward))は、計算された平均報酬と標準偏差を表示します。これにより、ユーザーはエージェントの性能を確認することができます。

agent.set_env(eval_env)
mean_reward, std_reward = sb3.common.evaluation.evaluate_policy(agent, agent.get_env(), n_eval_episodes=30)

print("Mean Reward: {} +/- {}".format(mean_reward, std_reward))

In [ ]:
agent.save(MODEL_PATH)

エージェントの評価

In [ ]:
agent.predict(training_env.reset())

In [ ]:
def testagent(observation, config):
    import numpy as np
    observation = np.array(observation['board']).reshape(1, config.rows, config.columns)
    action, _ = agent.predict(observation)
    return int(action)

In [ ]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds // 2)
    # Agent 2 goes first (roughly) half the time
    outcomes += [[b, a] for [a, b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds - n_rounds // 2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1, -1]) / len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1, 1] ) /len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [ ]:
get_win_percentages(agent1=testagent, agent2="negamax")

In [ ]:
get_win_percentages(agent1=testagent, agent2="random")

In [ ]:
env = make("connectx", debug=True)

# Two random agents play one game round
env.run([testagent, "negamax"])

# Show the game
env.render(mode="ipython")

In [ ]:
agent = sb3.PPO.load(MODEL_PATH)

In [ ]:
agent.policy.state_dict().keys()

In [ ]:
env = make("connectx", debug=True)
env.run([agent, agent])